# Configuration Management & Experiment Tracking Demo

This notebook demonstrates the configuration management system and experiment tracking features.

**Note:** This is optional reference material. The main project workflow is in `Project4-Semantic-Segmentation.ipynb`.

In [ ]:
# Configuration Management & Experiment Tracking Setup

print("="*70)
print("CONFIGURATION MANAGEMENT SYSTEM")
print("="*70)
print("""
This section demonstrates using YAML configs for easy model management
and performance tracking across different experiments.

Benefits:
- Store different model configurations (baseline, large, fast, etc.)
- Easy switching between configurations
- Automatic performance logging
- Track metrics and generate analysis reports
""")

# Create sample config files if they don't exist
from src.config_loader import create_sample_configs, ConfigLoader, PerformanceTracker

config_dir = create_sample_configs()
print(f"\n✓ Sample configs created in: {config_dir}")
print("\nAvailable configs:")
for cfg_file in sorted(config_dir.glob("*.yaml")):
    print(f"  - {cfg_file.name}")

In [ ]:
# Load Configuration from YAML

# Choose which config to use
config_name = "config_baseline.yaml"  # Change to other configs for experiments
config_path = f"./configs/{config_name}"

# Load configuration
experiment_config = ConfigLoader.load_experiment_config(config_path)

print(f"Loaded Configuration: {config_name}")
print(f"Experiment Name: {experiment_config.experiment_name}")
print(f"\nModel Configuration:")
print(f"  Model: {experiment_config.model.model_name}")
print(f"  Classes: {experiment_config.model.num_classes}")
print(f"\nTraining Configuration:")
print(f"  Learning Rate: {experiment_config.training.learning_rate}")
print(f"  Max Iterations: {experiment_config.training.max_iter}")
print(f"  Batch Size: {experiment_config.training.batch_size}")
print(f"  Image Size: {experiment_config.data.image_size}")

# You can also create a custom config
# experiment_config = ExperimentConfig(
#     experiment_name="my_custom_experiment",
#     model=ModelConfig(...),
#     training=TrainingConfig(...),
#     data=DataConfig(...)
# )

print("\n✓ Configuration loaded successfully!")

In [ ]:
# Initialize Performance Tracker

# Create a tracker for this experiment
tracker = PerformanceTracker(
    output_dir=experiment_config.output_dir,
    experiment_name=experiment_config.experiment_name
)

print(f"Performance Tracker initialized")
print(f"Metrics will be saved to: {tracker.metrics_dir}")
print(f"\nTracker Features:")
print("  - Log metrics per iteration (epoch)")
print("  - Log metrics per batch (for training)")
print("  - Save results to JSON and CSV")
print("  - Generate summary report")
print("  - Plot metrics over training")
print("  - Find and report best metrics")

# Save the configuration to the output directory
output_config_path = tracker.metrics_dir / "config.yaml"
ConfigLoader.save_config(experiment_config, str(output_config_path))
print(f"\n✓ Configuration saved to: {output_config_path}")

In [ ]:
# Example: Logging Metrics During Training
import numpy as np

print("="*70)
print("EXAMPLE: Logging Metrics During Training")
print("="*70)
print("""
Here's how to use the tracker during your training loop:

During training, log metrics like this:

    for epoch in range(num_epochs):
        # Training phase
        for batch_idx, (images, masks) in enumerate(train_loader):
            loss = train_step(...)

            # Log batch metrics (optional, for monitoring)
            if batch_idx % 10 == 0:
                tracker.log_batch(epoch, batch_idx, {'loss': loss})

        # Validation phase
        val_metrics = validate(model, val_loader)
        tracker.log_iteration(epoch, val_metrics, stage='val')

After training, save results:

    tracker.save_metrics_json()   # Save to JSON
    tracker.save_metrics_csv()    # Save to CSV
    tracker.save_summary()        # Generate text summary
    tracker.plot_metrics()        # Plot metrics
""")

# Example: Simulate logging some dummy metrics
print("\nSimulating a training run with dummy metrics...")

# Simulate 5 epochs
num_epochs = 5
for epoch in range(num_epochs):
    # Simulate training loss (decreasing)
    train_loss = 1.0 - (epoch * 0.15) + np.random.normal(0, 0.05)
    tracker.log_iteration(epoch, {'loss': train_loss}, stage='train')

    # Simulate validation metrics (improving)
    val_loss = 0.9 - (epoch * 0.12) + np.random.normal(0, 0.08)
    val_iou = 0.3 + (epoch * 0.12) + np.random.normal(0, 0.02)
    val_dice = 0.35 + (epoch * 0.13) + np.random.normal(0, 0.02)

    tracker.log_iteration(
        epoch,
        {
            'loss': val_loss,
            'iou': val_iou,
            'dice': val_dice,
        },
        stage='val'
    )

print(f"✓ Logged {len(tracker.metrics_history)} metric records")
print(f"\nMetric history sample (last 3 records):")
for record in tracker.metrics_history[-3:]:
    print(f"  {record}")

In [ ]:
# Save Metrics and Generate Analysis Reports

print("="*70)
print("SAVING METRICS AND GENERATING REPORTS")
print("="*70)

# Save metrics in different formats
json_file = tracker.save_metrics_json()
csv_file = tracker.save_metrics_csv()
summary_file = tracker.save_summary()

print(f"\n✓ Saved metrics to:")
print(f"  JSON: {json_file}")
print(f"  CSV:  {csv_file}")
print(f"  Summary: {summary_file}")

# Read and display summary
print("\n" + "="*70)
print("TRAINING SUMMARY")
print("="*70)
with open(summary_file) as f:
    print(f.read())

# Find best metrics
print("\n" + "="*70)
print("BEST METRICS")
print("="*70)

best_val_iou = tracker.get_best_metrics('iou', stage='val')
best_val_dice = tracker.get_best_metrics('dice', stage='val')

if best_val_iou:
    print(f"\nBest Validation IoU:")
    print(f"  Iteration: {best_val_iou.get('iteration')}")
    print(f"  IoU Score: {best_val_iou.get('iou'):.6f}")

if best_val_dice:
    print(f"\nBest Validation Dice:")
    print(f"  Iteration: {best_val_dice.get('iteration')}")
    print(f"  Dice Score: {best_val_dice.get('dice'):.6f}")

print("\n✓ Analysis complete!")

In [ ]:
# Plot Metrics Over Training

print("="*70)
print("VISUALIZATION: Metrics Over Training")
print("="*70)

# Plot selected metrics
tracker.plot_metrics(metrics_to_plot=['loss', 'iou', 'dice'])

print("\n✓ Metrics plotted and saved!")

In [ ]:
# Experiment Comparison Workflow
from pathlib import Path
import json
import pandas as pd

print("="*70)
print("COMPARING MULTIPLE EXPERIMENTS")
print("="*70)
print("""
Workflow for comparing different configurations:

1. Run experiment with config_baseline.yaml
   - Train model
   - Track metrics with PerformanceTracker
   - Save results

2. Run experiment with config_large_model.yaml
   - Same steps, different config
   - Results saved in separate directory

3. Compare Results:
""")

# Example comparison
def compare_experiments(experiment_dirs):
    """Compare metrics across multiple experiments"""
    results = {}

    for exp_dir in experiment_dirs:
        exp_path = Path(exp_dir)
        metrics_files = list(exp_path.glob("metrics/*.json"))

        if metrics_files:
            with open(metrics_files[0]) as f:
                metrics = json.load(f)

            # Extract best validation metrics
            val_records = [m for m in metrics if m.get('stage') == 'val']
            if val_records:
                results[exp_path.name] = {
                    'best_iou': max([m.get('iou', 0) for m in val_records]),
                    'best_dice': max([m.get('dice', 0) for m in val_records]),
                }

    return pd.DataFrame(results).T

# Example: Create comparison table (with dummy data)
comparison_data = {
    'baseline': {'best_iou': 0.65, 'best_dice': 0.72},
    'large_model': {'best_iou': 0.70, 'best_dice': 0.76},
    'fast_train': {'best_iou': 0.58, 'best_dice': 0.64},
}

comparison_df = pd.DataFrame(comparison_data).T
print("\nExample Experiment Comparison:")
print(comparison_df)
print("\n✓ Use this workflow to track and compare all your experiments!")